# Sistema utilizado:
Foi usado o programa 7z para o desenvolvimento das seguintes questões.
Para a execução, foi utilizado 4 filmes totalizando 6.2GB para compressão.
E para monitorar o sistema, foi usado os comandos VMSTAT e IOSTAT.

# Utilização Média da CPU e Disco.

In [85]:
import pandas as pd
import numpy as np

def busyTime(us):
    count  = 0
    busyTimeList = []
    for i in us:
        if i > 10:
            count += 1
        elif count != 0:
            busyTimeList.append(count)
            count = 0

    return busyTimeList


df = pd.read_csv('resultCPUVMSTAT.csv')
df['time'] = pd.to_datetime(df['time'])

T = df['time'][len(df) - 1] - df['time'][0]
T = T.total_seconds() # tempo de execução da compressão

btlist = busyTime(df['us'])
busyTimeCPU = sum(btlist)

print('Total time: {}s.'.format(T))

# FEITO A MEDIA DAS UTILIZAÇÔES DA CPU DENTRO DO TEMPO DE EXECUÇÂO
#CPUutil1 = np.mean(df['us'])
#print('CPU Average Util: {}%.'.format(CPUutil1))


# Utilizando que U = B / T sendo B o tempo que a CPU passou ocupada e T o tempo total de execução do sistema.
CPUutil2 = (busyTimeCPU / T) * 100
print('CPU Average Util: {}%.'.format(CPUutil2))

df1 = pd.read_csv('resultDisk.csv') 

# FEITO A MEDIA DAS UTILIZAÇÔES DO DISCO DENTRO DO TEMPO DE EXECUÇÂO
Diskutil = np.mean(df1['%util'])
print('Disk Average Util: {}%.'.format(Diskutil))


Total time: 385.0s.
CPU Average Util: 68.57142857142857%.
Disk Average Util: 52.33287564766842%.


# Demanda Média da CPU, Disco E Throughput do Disco:

In [86]:
Xdfdisk = df1['r/s'] + df1['w/s']
Xdisk = np.mean(Xdfdisk)

Sdfdisk = df1['w_await'] + df1['r_await']
Sdisk1 = np.mean(Sdfdisk) / 1000
#print(Sdisk1)

diskDemand = (Diskutil / 100) * Sdisk1

print('Disk Demand: {}.'.format(diskDemand))
print('Disk Throughput: {}.'.format(Xdisk))

Xsystem = (Diskutil / 100) / diskDemand

# Uk = X * Dk
#CPUDemand = (CPUutil1 / 100) / Xsystem
#print('CPU Demand : {}.'.format(CPUDemand))

#Uk = X * Dk
CPUDemand2 = (CPUutil2 / 100) / Xsystem
print('CPU Demand : {}.'.format(CPUDemand2))

Disk Demand: 0.5058617508607076.
Disk Throughput: 74.80575129533679.
CPU Demand : 0.6628273811991113.


# Letra D)

In [88]:
# 1 instancia a cada 10 segundos
# N* = (D + Z) / Dmax
numberOfInstances = (((CPUDemand2 + diskDemand)) + 10)/ max(CPUDemand2,diskDemand)

print('# of Instances simultaneously : {} .'.format(numberOfInstances))

# of Instances simultaneously : 16.850072053231578 .
